In [1]:
# Import libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from scipy.stats import chi2_contingency

In [2]:
# Load data and view first 5 rows
data = pd.read_stata('rainfall_2.dta')
data.head()

,fid,cell_count,mean_mon_rainfall_mm,time,iso3,svycode,svyl1cd,svyl1nm,svyl2cd,svyl2nm,prttynm,rn
0,0,6,52.755573,1981.01001,UGA,uga2013,1,Central,102,KAMPALA,KAMPALA,
1,1,80,20.932796,1981.01001,UGA,uga2013,1,Central,123,KYANKWANZI,KIBOGA,
2,2,17,33.781631,1981.01001,UGA,uga2013,1,Central,118,BUKOMANSIMBI,MASAKA,
3,3,103,38.626671,1981.01001,UGA,uga2013,1,Central,124,LWENGO,MASAKA,
4,4,27,37.377052,1981.01001,UGA,uga2013,1,Central,122,KALUNGU,MASAKA,


In [3]:
# Check dataframe information
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 193004 entries, 0 to 193003
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   fid                   193004 non-null  int16  
 1   cell_count            193004 non-null  int16  
 2   mean_mon_rainfall_mm  193004 non-null  float32
 3   time                  193004 non-null  float32
 4   iso3                  193004 non-null  object 
 5   svycode               193004 non-null  object 
 6   svyl1cd               193004 non-null  int8   
 7   svyl1nm               193004 non-null  object 
 8   svyl2cd               193004 non-null  int16  
 9   svyl2nm               193004 non-null  object 
 10  prttynm               193004 non-null  object 
 11  rn                    193004 non-null  object 
dtypes: float32(2), int16(3), int8(1), object(6)
memory usage: 13.1+ MB


In [4]:
# View unique country iso3 codes
data.iso3.unique()

array(['UGA', 'TZA', 'GHA'], dtype=object)

In [5]:
# Load Ghana data into 'gh_rain_data'
gh_rain_data = data[data['iso3'] == 'GHA']
# View first 5 rows
gh_rain_data.head()

,fid,cell_count,mean_mon_rainfall_mm,time,iso3,svycode,svyl1cd,svyl1nm,svyl2cd,svyl2nm,prttynm,rn
282,282,19,6.065213,1981.01001,GHA,gha2012,6,Ashanti,17,Ahafo Ano North,Ahafo Ano North,0
283,283,39,5.155084,1981.01001,GHA,gha2012,6,Ashanti,16,Ahafo Ano South,Ahafo Ano South,1
284,284,41,4.253942,1981.01001,GHA,gha2012,6,Ashanti,2,Amansie West,Amansie West,2
285,285,49,3.583445,1981.01001,GHA,gha2012,6,Ashanti,10,Asante Akim North,Asante Akim North,3
286,286,37,4.436135,1981.01001,GHA,gha2012,6,Ashanti,9,Asante Akim South,Asante Akim South,4


In [6]:
gh_rain_data.shape

(72590, 12)

In [7]:
# View last 5 rows
gh_rain_data.tail()

,fid,cell_count,mean_mon_rainfall_mm,time,iso3,svycode,svyl1cd,svyl1nm,svyl2cd,svyl2nm,prttynm,rn
192999,447,36,171.675644,2016.069946,GHA,gha2012,4,Volta,13,Jasikan,Jasikan,165
193000,448,13,52.204678,2016.069946,GHA,gha2012,4,Volta,4,Ketu North,Ketu North,166
193001,449,10,44.619186,2016.069946,GHA,gha2012,4,Volta,3,Ketu South,Ketu South,167
193002,450,31,238.490906,2016.069946,GHA,gha2012,4,Volta,18,Nkwanta North,Nkwanta North,168
193003,451,100,255.096298,2016.069946,GHA,gha2012,4,Volta,17,Nkwanta South,Nkwanta South,169


In [8]:
# View statistical evaluations of numerical columns
gh_rain_data.describe()

,fid,cell_count,mean_mon_rainfall_mm,time,svyl1cd,svyl2cd
count,72590.000000,72590.000000,72590.000000,72590.000000,72590.000000,72590.000000
mean,366.500000,46.241176,101.957565,1998.808594,5.323529,9.964706
std,49.074262,49.265356,79.951988,10.282920,2.577301,6.259055
min,282.000000,2.000000,0.251909,1981.010010,1.000000,1.000000
25%,324.000000,18.000000,31.121050,1989.109985,3.000000,5.000000
50%,366.500000,31.000000,91.322670,1998.099976,6.000000,9.000000
75%,409.000000,54.000000,155.852806,2007.089966,7.000000,15.000000
max,451.000000,287.000000,579.755920,2016.069946,10.000000,27.000000


In [9]:
# Number of duplicated rows
gh_rain_data.duplicated().sum()

0

In [10]:
# Number of null values in each column
gh_rain_data.isna().sum()

fid                     0
cell_count              0
mean_mon_rainfall_mm    0
time                    0
iso3                    0
svycode                 0
svyl1cd                 0
svyl1nm                 0
svyl2cd                 0
svyl2nm                 0
prttynm                 0
rn                      0
dtype: int64

In [11]:
# Unique time (year + month) values
print("There are", gh_rain_data.time.unique().size, "values in the time column")
gh_rain_data.time.unique()

There are 427 values in the time column


array([1981.01, 1981.02, 1981.03, 1981.04, 1981.05, 1981.06, 1981.07,
       1981.08, 1981.09, 1981.1 , 1981.11, 1981.12, 1982.01, 1982.02,
       1982.03, 1982.04, 1982.05, 1982.06, 1982.07, 1982.08, 1982.09,
       1982.1 , 1982.11, 1982.12, 1983.01, 1983.02, 1983.03, 1983.04,
       1983.05, 1983.06, 1983.07, 1983.08, 1983.09, 1983.1 , 1983.11,
       1983.12, 1984.01, 1984.02, 1984.03, 1984.04, 1984.05, 1984.06,
       1984.07, 1984.08, 1984.09, 1984.1 , 1984.11, 1984.12, 1985.01,
       1985.02, 1985.03, 1985.04, 1985.05, 1985.06, 1985.07, 1985.08,
       1985.09, 1985.1 , 1985.11, 1985.12, 1986.01, 1986.02, 1986.03,
       1986.04, 1986.05, 1986.06, 1986.07, 1986.08, 1986.09, 1986.1 ,
       1986.11, 1986.12, 1987.01, 1987.02, 1987.03, 1987.04, 1987.05,
       1987.06, 1987.07, 1987.08, 1987.09, 1987.1 , 1987.11, 1987.12,
       1988.01, 1988.02, 1988.03, 1988.04, 1988.05, 1988.06, 1988.07,
       1988.08, 1988.09, 1988.1 , 1988.11, 1988.12, 1989.01, 1989.02,
       1989.03, 1989

In [12]:
# View unique regions
print("There are", gh_rain_data.svyl1nm.unique().size, "regions in the svyl1nm column")
gh_rain_data.svyl1nm.unique()

There are 10 regions in the svyl1nm column


array(['Ashanti', 'Brong Ahafo', 'Central', 'Eastern', 'Greater Accra',
       'Upper West', 'Volta', 'Western', 'Upper East', 'Northern'],
      dtype=object)

In [13]:
# View unique 'svycode' values
gh_rain_data.svycode.unique()

array(['gha2012'], dtype=object)

In [14]:
# View unique sorted 'svyl1cd' values
np.sort(gh_rain_data.svyl1cd.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int8)

In [15]:
# View unique sorted 'svyl2cd' values
np.sort(gh_rain_data.svyl2cd.unique())

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27], dtype=int16)

In [16]:
# View unique sorted 'svyl2nm' values
print("There are", gh_rain_data.svyl2nm.unique().size, "districts in the svyl2nm column")
np.sort(gh_rain_data.svyl2nm.unique())

There are 170 districts in the svyl2nm column


array(['A M A', 'Abura / Asebu / Kwamankese', 'Adaklu Anyigbe',
       'Adansi North', 'Adansi South', 'Adenta', 'Afigya Kwabre',
       'Agona East', 'Agona West', 'Ahafo Ano North', 'Ahafo Ano South',
       'Ahanta West', 'Ajumanku / Enyan / Essiam', 'Akatsi',
       'Akwapim North', 'Akwapim South', 'Akyemansa', 'Amansie Central',
       'Amansie West', 'Aowin / Suaman', 'Asante Akim North',
       'Asante Akim South', 'Ashaiman', 'Asikuma / Odoben / Brakwa',
       'Assin North', 'Assin South', 'Asunafo North', 'Asunafo South',
       'Asuogyaman', 'Asutifi', 'Atebubu Amantin', 'Atiwa',
       'Atwima Kwanwoma', 'Atwima Mponua', 'Atwima Nwabiagya',
       'Awutu Senya', 'Bawku Municipal', 'Bawku West', 'Bekwai Municipal',
       'Berekum', 'Bia', 'Biakoye', 'Birim Central Municipal',
       'Birim North', 'Birim South', 'Bole', 'Bolgatanga', 'Bongo',
       'Bosome Freho', 'Bosumtwi', 'Builsa', 'Bunkpurugu Yonyo',
       'Cape Coast', 'Central Gonja', 'Chereponi', 'Dangbe East',
 

In [17]:
# View unique sorted 'prttynm' values
print("There are", gh_rain_data.prttynm.unique().size, "districts in the prttynm column")
np.sort(gh_rain_data.prttynm.unique())

There are 170 districts in the prttynm column


array(['A M a', 'Abura / Asebu / Kwamankese', 'Adaklu Anyigbe',
       'Adansi North', 'Adansi South', 'Adenta', 'Afigya Kwabre',
       'Agona East', 'Agona West', 'Ahafo Ano North', 'Ahafo Ano South',
       'Ahanta West', 'Ajumanku / Enyan / Essiam', 'Akatsi',
       'Akwapim North', 'Akwapim South', 'Akyemansa', 'Amansie Central',
       'Amansie West', 'Aowin / Suaman', 'Asante Akim North',
       'Asante Akim South', 'Ashaiman', 'Asikuma / Odoben / Brakwa',
       'Assin North', 'Assin South', 'Asunafo North', 'Asunafo South',
       'Asuogyaman', 'Asutifi', 'Atebubu Amantin', 'Atiwa',
       'Atwima Kwanwoma', 'Atwima Mponua', 'Atwima Nwabiagya',
       'Awutu Senya', 'Bawku Municipal', 'Bawku West', 'Bekwai Municipal',
       'Berekum', 'Bia', 'Biakoye', 'Birim Central Municipal',
       'Birim North', 'Birim South', 'Bole', 'Bolgatanga', 'Bongo',
       'Bosome Freho', 'Bosumtwi', 'Builsa', 'Bunkpurugu Yonyo',
       'Cape Coast', 'Central Gonja', 'Chereponi', 'Dangbe East',
 

In [18]:
# View unique 'rn' values
gh_rain_data.rn.unique()

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56',
       '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67',
       '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78',
       '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89',
       '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100',
       '101', '102', '103', '104', '105', '106', '107', '108', '109',
       '110', '111', '112', '113', '114', '115', '116', '117', '118',
       '119', '120', '121', '122', '123', '124', '125', '126', '127',
       '128', '129', '130', '131', '132', '133', '134', '135', '136',
       '137', '138', '139', '140', '141', '142', '143', '144

In [19]:
# Number of fields per column where svyl2nm and prttynm aren't the same/equal
gh_rain_data[gh_rain_data['svyl2nm'] != gh_rain_data['prttynm']].count()

fid                     854
cell_count              854
mean_mon_rainfall_mm    854
time                    854
iso3                    854
svycode                 854
svyl1cd                 854
svyl1nm                 854
svyl2cd                 854
svyl2nm                 854
prttynm                 854
rn                      854
dtype: int64

In [20]:
# svyl2nm values where svyl2nm and prttynm aren't equal
gh_rain_data[gh_rain_data['svyl2nm'] != gh_rain_data['prttynm']].svyl2nm.value_counts()

K M A    427
A M A    427
Name: svyl2nm, dtype: int64

In [21]:
# prttynm values where svyl2nm and prttynm aren't equal
gh_rain_data[gh_rain_data['svyl2nm'] != gh_rain_data['prttynm']].prttynm.value_counts()

A M a    427
K M a    427
Name: prttynm, dtype: int64

In [22]:
# Create new dataframe with selected columns
df = gh_rain_data[['cell_count', 'mean_mon_rainfall_mm', 'time', 'svyl1nm', 'svyl2nm', 'rn', 'svyl1cd', 'svyl2cd']]
df.head()

,cell_count,mean_mon_rainfall_mm,time,svyl1nm,svyl2nm,rn,svyl1cd,svyl2cd
282,19,6.065213,1981.01001,Ashanti,Ahafo Ano North,0,6,17
283,39,5.155084,1981.01001,Ashanti,Ahafo Ano South,1,6,16
284,41,4.253942,1981.01001,Ashanti,Amansie West,2,6,2
285,49,3.583445,1981.01001,Ashanti,Asante Akim North,3,6,10
286,37,4.436135,1981.01001,Ashanti,Asante Akim South,4,6,9


In [23]:
# Create new year and month columns from the time column
year = []
month = []
for i in df['time']:
    y = int(i // 1)
    m = round((i % y) * 100)
    year.append(y)
    month.append(m)
df.loc[:, 'year'] = year
df.loc[:, 'month'] = month

/Users/doski/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/doski/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [24]:
print("There are", df.size, "rows in the dataset.")

There are 725900 rows in the dataset.


In [25]:
# View first five columns of the new dataframe
df.head()

,cell_count,mean_mon_rainfall_mm,time,svyl1nm,svyl2nm,rn,svyl1cd,svyl2cd,year,month
282,19,6.065213,1981.01001,Ashanti,Ahafo Ano North,0,6,17,1981,1
283,39,5.155084,1981.01001,Ashanti,Ahafo Ano South,1,6,16,1981,1
284,41,4.253942,1981.01001,Ashanti,Amansie West,2,6,2,1981,1
285,49,3.583445,1981.01001,Ashanti,Asante Akim North,3,6,10,1981,1
286,37,4.436135,1981.01001,Ashanti,Asante Akim South,4,6,9,1981,1


In [26]:
# View the last five columns of the new dataframe
df.tail()

,cell_count,mean_mon_rainfall_mm,time,svyl1nm,svyl2nm,rn,svyl1cd,svyl2cd,year,month
192999,36,171.675644,2016.069946,Volta,Jasikan,165,4,13,2016,7
193000,13,52.204678,2016.069946,Volta,Ketu North,166,4,4,2016,7
193001,10,44.619186,2016.069946,Volta,Ketu South,167,4,3,2016,7
193002,31,238.490906,2016.069946,Volta,Nkwanta North,168,4,18,2016,7
193003,100,255.096298,2016.069946,Volta,Nkwanta South,169,4,17,2016,7


In [27]:
# View the unique years in the dataframe
print("The dataframe contains data for", df.year.unique().size, "years")
df.year.unique()

The dataframe contains data for 36 years


array([1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991,
       1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016])

In [28]:
# View the unique months in the dataframe 
df.month.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [29]:
# View the number of times each year appears in the dataframe
df.year.value_counts()

1983    2040
1982    2040
1999    2040
1998    2040
1997    2040
1996    2040
1995    2040
1994    2040
1993    2040
1992    2040
1991    2040
1990    2040
1989    2040
1988    2040
1987    2040
1986    2040
1985    2040
2000    2040
2001    2040
2002    2040
2003    2040
1981    2040
2015    2040
2014    2040
2013    2040
2012    2040
2011    2040
2010    2040
2009    2040
2008    2040
2007    2040
2006    2040
2005    2040
2004    2040
1984    2040
2016    1190
Name: year, dtype: int64

In [30]:
# View the number of times each month appears in the dataframe
df.month.value_counts()

7     6120
6     6120
5     6120
4     6120
3     6120
2     6120
1     6120
12    5950
11    5950
10    5950
9     5950
8     5950
Name: month, dtype: int64

In [31]:
# View the number of times each region appears in the dataframe
df.svyl1nm.value_counts()

Ashanti          11529
Brong Ahafo       9394
Eastern           8967
Northern          8540
Volta             7686
Central           7259
Western           7259
Greater Accra     4270
Upper West        3843
Upper East        3843
Name: svyl1nm, dtype: int64

In [32]:
# Create new dataframe with selected columns
df = df[['svyl1nm', 'svyl2nm', 'year', 'month', 'mean_mon_rainfall_mm']]
df = df.rename(columns={"svyl1nm":"region", "svyl2nm":"district"})
df.head()

,region,district,year,month,mean_mon_rainfall_mm
282,Ashanti,Ahafo Ano North,1981,1,6.065213
283,Ashanti,Ahafo Ano South,1981,1,5.155084
284,Ashanti,Amansie West,1981,1,4.253942
285,Ashanti,Asante Akim North,1981,1,3.583445
286,Ashanti,Asante Akim South,1981,1,4.436135


In [33]:
region_district_df = df.groupby(['region','district']).size().reset_index(name='Count')

In [34]:
# List all districts per region
for val in list(region_district_df['region'].unique()):
    districts = region_district_df.query('region == @val')['district'].unique()
    print(" Region: {}, Number of Districts: {}\n{}\n".format(val, len(districts),districts))


 Region: Ashanti, Number of Districts: 27
['Adansi North' 'Adansi South' 'Afigya Kwabre' 'Ahafo Ano North'
 'Ahafo Ano South' 'Amansie Central' 'Amansie West' 'Asante Akim North'
 'Asante Akim South' 'Atwima Kwanwoma' 'Atwima Mponua' 'Atwima Nwabiagya'
 'Bekwai Municipal' 'Bosome Freho' 'Bosumtwi' 'Ejisu Juaben'
 'Ejura Sekye Dumasi' 'K M A' 'Kwabre East' 'Mampong Municipal'
 'Obuasi Municipal' 'Offinso Municipal' 'Offinso North'
 'Sekyere Afram Plains' 'Sekyere Central' 'Sekyere East' 'Sekyere South']

 Region: Brong Ahafo, Number of Districts: 22
['Asunafo North' 'Asunafo South' 'Asutifi' 'Atebubu Amantin' 'Berekum'
 'Dormaa East' 'Dormaa Municipal' 'Jaman North' 'Jaman South' 'Kintampo'
 'Kintampo South' 'Nkoranza North' 'Nkoranza South' 'Pru' 'Sene'
 'Sunyani Municipal' 'Sunyani West' 'Tain' 'Tano North' 'Tano South'
 'Techiman' 'Wenchi']

 Region: Central, Number of Districts: 17
['Abura / Asebu / Kwamankese' 'Agona East' 'Agona West'
 'Ajumanku / Enyan / Essiam' 'Asikuma / Odoben

In [35]:
# Read flood start date data
un = pd.read_csv("flood/najibi_devineni_esd2018_start_date_part1.csv")
print("The shape of the dataset is", un.shape)
un.head()

The shape of the dataset is (293, 31)


,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,19850101.0,19860101.0,19870101.0,19880101.0,19890101.0,19900103.0,19910105.0,19920102.0,19930101.0,19940104.0,...,20060101.0,20070101.0,20080102.0,20090103.0,20100101.0,20110101.0,20120101.0,20130103.0,20140101.0,20150101.0
1,19850115.0,19860106.0,19870115.0,19880130.0,19890101.0,19900109.0,19910127.0,19920104.0,19930103.0,19940109.0,...,20060102.0,20070101.0,20080103.0,20090108.0,20100101.0,20110101.0,20120101.0,20130103.0,20140108.0,20150105.0
2,19850120.0,19860201.0,19870120.0,19880201.0,19890102.0,19900114.0,19910128.0,19920104.0,19930105.0,19940112.0,...,20060103.0,20070101.0,20080105.0,20090108.0,20100101.0,20110101.0,20120117.0,20130103.0,20140110.0,20150105.0
3,19850204.0,19860212.0,19870126.0,19880201.0,19890114.0,19900114.0,19910202.0,19920105.0,19930106.0,19940116.0,...,20060108.0,20070103.0,20080107.0,20090120.0,20100104.0,20110105.0,20120122.0,20130104.0,20140115.0,20150105.0
4,19850209.0,19860327.0,19870128.0,19880201.0,19890117.0,19900120.0,19910224.0,19920105.0,19930115.0,19940116.0,...,20060108.0,20070104.0,20080108.0,20090122.0,20100110.0,20110107.0,20120122.0,20130104.0,20140115.0,20150201.0


In [36]:
un.size

9083

In [37]:
# Read flood stop date data
deux = pd.read_csv("flood/najibi_devineni_esd2018_end_date_part2.csv")
print("The shape of the dataset is", deux.shape)
deux.head()

The shape of the dataset is (293, 31)


,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,19850105.0,19860106.0,19870126.0,19880102.0,19890108.0,19900114.0,19910106.0,19920103.0,19930103.0,19940108.0,...,20060104.0,20070301.0,20080106.0,20090113.0,20100113.0,20110204.0,20120109.0,20130110.0,20140104.0,20150209.0
1,19850202.0,19860403.0,19870205.0,19880205.0,19890102.0,19900111.0,19910205.0,19920106.0,19930106.0,19940113.0,...,20060120.0,20070112.0,20080109.0,20090116.0,20100113.0,20110118.0,20120109.0,20130110.0,20140207.0,20150209.0
2,19850121.0,19860501.0,19870130.0,19880210.0,19890108.0,19900115.0,19910217.0,19920105.0,19930105.0,19940122.0,...,20060111.0,20070104.0,20080109.0,20090116.0,20100110.0,20110118.0,20120214.0,20130110.0,20140501.0,20150209.0
3,19850218.0,19860227.0,19870128.0,19880204.0,19890117.0,19900314.0,19910208.0,19920108.0,19930120.0,19940120.0,...,20060214.0,20070310.0,20080116.0,20090127.0,20100113.0,20110118.0,20120307.0,20130110.0,20140207.0,20150111.0
4,19850211.0,19860405.0,19870214.0,19880202.0,19890127.0,19900201.0,19910301.0,19920107.0,19930118.0,19940120.0,...,20060112.0,20070122.0,20080115.0,20090129.0,20100128.0,20110110.0,20120206.0,20130110.0,20140207.0,20150209.0


In [38]:
# Read flood latitude data
trois = pd.read_csv("flood/najibi_devineni_esd2018_latitude_part3.csv")
print("The shape of the dataset is", trois.shape)
trois.head()

The shape of the dataset is (293, 31)


,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,35.81420,44.0047,-19.6111,21.38250,10.5704,7.60118,53.58930,57.2800,-17.83680,13.43580,...,-8.02272,-3.13897,-6.22483,-17.12710,-17.8267,-31.05530,-19.46370,9.88586,-12.4186,-18.7946
1,-18.71110,-16.5525,-24.1520,9.60268,-28.4838,45.01350,31.72080,12.5093,27.24470,-9.38978,...,41.55210,-22.41550,-28.44270,-16.59640,-30.9508,-18.73870,8.77228,5.48006,-6.6324,-19.6675
2,10.02070,-16.1155,-16.7537,-22.49390,-21.2427,13.44730,31.65190,-22.5599,7.62904,-6.83998,...,47.71710,15.09750,39.63080,42.24830,-21.8476,8.13264,-21.45130,-3.86702,-13.3888,-16.4638
3,1.01489,39.2816,-23.5772,30.64730,57.2883,-26.69470,-1.31926,18.2253,33.01910,-9.05287,...,6.89182,-17.46670,6.90584,7.12112,41.4664,7.60326,-21.71760,-9.28298,-20.2044,-19.1872
4,-25.86930,45.0248,43.1418,19.32100,50.9569,33.21400,54.90630,-31.3961,56.31410,-25.26280,...,-6.38134,4.96798,40.77670,-2.63531,-22.9804,50.34680,-17.78400,7.25699,-17.2057,41.4571


In [39]:
# Read flood longitude data
quart = pd.read_csv("flood/najibi_devineni_esd2018_longitude_part4.csv")
print("The shape of the dataset is", quart.shape)
quart.head()

The shape of the dataset is (293, 31)


,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,5.23026,-116.9540,47.4609,-157.7820,-66.68460,80.82280,-7.83800,-3.74148,178.00100,123.4500,...,113.3420,30.3995,106.8280,143.1360,-63.6012,28.40700,-43.4893,118.6260,-71.5232,35.3564
1,-45.34890,-66.0628,-60.2051,-82.9868,30.32500,-123.09400,58.57150,53.90830,56.20390,38.4373,...,26.4726,-44.0032,152.8700,-44.8911,149.0580,-43.44900,99.4238,101.6610,107.7060,45.9218
2,122.97400,-69.0747,-64.8170,-43.2761,165.50300,144.77800,64.36780,-43.32070,-76.36940,107.6570,...,19.5834,43.8761,-119.1970,-2.6099,-43.7619,125.56600,34.0721,121.8830,-64.0135,-63.7661
3,124.60600,-121.3800,-46.6809,-90.2200,-4.50437,-56.65930,-79.65330,-66.45260,-116.87600,148.5300,...,-58.6627,35.1463,81.7574,124.9580,20.0828,81.25220,146.6460,-76.4573,28.9231,127.0560
4,32.34910,-87.8814,41.8577,-70.4315,-3.96448,9.45159,-1.58219,-61.72820,-3.70463,151.6850,...,108.2150,115.9070,-86.6978,120.2640,-44.0681,7.63154,178.0760,125.3080,30.8531,26.5972


In [40]:
# Read flood damages data
cinq = pd.read_csv("flood/najibi_devineni_esd2018_damages_part5.csv")
print("The shape of the dataset is", cinq.shape)
cinq.head()

The shape of the dataset is (293, 31)


,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,NaN,NaN,NaN,28000000.0,NaN,NaN,NaN,NaN,64000000.0,NaN,...,NaN,0.0,NaN,4000000.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2.000000e+09,42000000.0,NaN,7000000.0,NaN,7200000.0,28000000.0,NaN,NaN,NaN,...,0.0,125000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,50000000.0,NaN,10000000.0,350000000.0,NaN,NaN,NaN,NaN,1900000.0,...,NaN,0.0,100000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,319000000.0,NaN,NaN,NaN,NaN,2000000.0,200000000.0,164000000.0,NaN,...,165000000.0,71000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.000000e+06,200000.0,583000000.0,NaN,NaN,233000000.0,NaN,NaN,NaN,NaN,...,245000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# Create new dataframe for flood data
flood = pd.DataFrame(columns=['StartDate', 'EndDate', 'Longitude', 'Latitude', 'Damages'])
flood.head()

,StartDate,EndDate,Longitude,Latitude,Damages


In [42]:
un.loc[1][30]

20150105.0

In [43]:
# Fill flood dataframe with data from previous separated flood dataframes
for i in range(31):
    for j in range(293):
        start = un.loc[j][i]
        end = deux.loc[j][i]
        long = quart.loc[j][i]
        lat = trois.loc[j][i]
        dam = cinq.loc[j][i]
        
        flood = flood.append({'StartDate':start, 'EndDate':end, 'Longitude':long, 'Latitude':lat, 'Damages':dam}, 
                     ignore_index=True)
flood

,StartDate,EndDate,Longitude,Latitude,Damages
0,19850101.0,19850105.0,5.23026,35.81420,NaN
1,19850115.0,19850202.0,-45.34890,-18.71110,2.000000e+09
2,19850120.0,19850121.0,122.97400,10.02070,NaN
3,19850204.0,19850218.0,124.60600,1.01489,NaN
4,19850209.0,19850211.0,32.34910,-25.86930,3.000000e+06
...,...,...,...,...,...
9078,NaN,NaN,NaN,NaN,NaN
9079,NaN,NaN,NaN,NaN,NaN
9080,NaN,NaN,NaN,NaN,NaN
9081,NaN,NaN,NaN,NaN,NaN


In [44]:
# Flood dataframe info
flood.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9083 entries, 0 to 9082
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   StartDate  4311 non-null   float64
 1   EndDate    4311 non-null   float64
 2   Longitude  4311 non-null   float64
 3   Latitude   4311 non-null   float64
 4   Damages    1272 non-null   float64
dtypes: float64(5)
memory usage: 354.9 KB


In [45]:
# Basic statistical analysis
flood.describe()

,StartDate,EndDate,Longitude,Latitude,Damages
count,4.311000e+03,4.311000e+03,4311.000000,4311.000000,1.272000e+03
mean,2.001674e+07,2.001691e+07,35.903480,18.553095,6.300971e+08
std,7.758352e+04,7.765554e+04,79.499852,22.734921,6.946908e+09
min,1.985010e+07,1.985010e+07,-173.454000,-52.825600,0.000000e+00
25%,1.996061e+07,1.996063e+07,-11.499700,5.749205,0.000000e+00
50%,2.003020e+07,2.003022e+07,44.168600,21.846200,1.000000e+07
75%,2.007121e+07,2.007122e+07,105.105500,35.951900,9.500000e+07
max,2.015122e+07,2.016013e+07,179.651000,68.159100,2.100000e+11


In [46]:
# First five rows of flood dataframe
flood.head()

,StartDate,EndDate,Longitude,Latitude,Damages
0,19850101.0,19850105.0,5.23026,35.81420,NaN
1,19850115.0,19850202.0,-45.34890,-18.71110,2.000000e+09
2,19850120.0,19850121.0,122.97400,10.02070,NaN
3,19850204.0,19850218.0,124.60600,1.01489,NaN
4,19850209.0,19850211.0,32.34910,-25.86930,3.000000e+06


In [47]:
# Last five rows of flood dataframe
flood.tail()

,StartDate,EndDate,Longitude,Latitude,Damages
9078,NaN,NaN,NaN,NaN,NaN
9079,NaN,NaN,NaN,NaN,NaN
9080,NaN,NaN,NaN,NaN,NaN
9081,NaN,NaN,NaN,NaN,NaN
9082,NaN,NaN,NaN,NaN,NaN


In [48]:
# Number of null fields in flood data
flood.isna().sum()

StartDate    4772
EndDate      4772
Longitude    4772
Latitude     4772
Damages      7811
dtype: int64

In [49]:
# Number of duplicated fields in flood data
flood.duplicated().sum()

4771

In [50]:
# View duplicated fields
flood[flood.duplicated()]

,StartDate,EndDate,Longitude,Latitude,Damages
70,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN
72,NaN,NaN,NaN,NaN,NaN
73,NaN,NaN,NaN,NaN,NaN
74,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
9078,NaN,NaN,NaN,NaN,NaN
9079,NaN,NaN,NaN,NaN,NaN
9080,NaN,NaN,NaN,NaN,NaN
9081,NaN,NaN,NaN,NaN,NaN


In [51]:
# Drop duplicated rows
flood = flood.drop_duplicates()

In [52]:
# Check the number of null fields
flood.isna().sum()

StartDate       1
EndDate         1
Longitude       1
Latitude        1
Damages      3040
dtype: int64

In [53]:
# View rows with null start date
flood[flood.StartDate.isna()]

,StartDate,EndDate,Longitude,Latitude,Damages
69,NaN,NaN,NaN,NaN,NaN


In [54]:
# Drop the above row
flood = flood.drop(labels=69, axis=0)

In [55]:
# View rows with null start date (Check if the row drop above was successfull)
flood[flood.StartDate.isna()]

,StartDate,EndDate,Longitude,Latitude,Damages


In [56]:
# Clean flood data info
flood.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4311 entries, 0 to 8890
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   StartDate  4311 non-null   float64
 1   EndDate    4311 non-null   float64
 2   Longitude  4311 non-null   float64
 3   Latitude   4311 non-null   float64
 4   Damages    1272 non-null   float64
dtypes: float64(5)
memory usage: 202.1 KB


In [57]:
# Basic statistical analysis on clean flood data
flood.describe()

,StartDate,EndDate,Longitude,Latitude,Damages
count,4.311000e+03,4.311000e+03,4311.000000,4311.000000,1.272000e+03
mean,2.001674e+07,2.001691e+07,35.903480,18.553095,6.300971e+08
std,7.758352e+04,7.765554e+04,79.499852,22.734921,6.946908e+09
min,1.985010e+07,1.985010e+07,-173.454000,-52.825600,0.000000e+00
25%,1.996061e+07,1.996063e+07,-11.499700,5.749205,0.000000e+00
50%,2.003020e+07,2.003022e+07,44.168600,21.846200,1.000000e+07
75%,2.007121e+07,2.007122e+07,105.105500,35.951900,9.500000e+07
max,2.015122e+07,2.016013e+07,179.651000,68.159100,2.100000e+11


In [58]:
# Create dataframe by filtering out Ghana's Latitudes
gh_flood = flood[flood['Latitude'] < 12]
gh_flood = gh_flood[gh_flood['Latitude'] > 4]
gh_flood.head()

,StartDate,EndDate,Longitude,Latitude,Damages
2,19850120.0,19850121.0,122.97400,10.02070,NaN
31,19850713.0,19850716.0,21.55090,11.67270,NaN
42,19850817.0,19850819.0,39.78510,11.05140,NaN
54,19850923.0,19850925.0,6.50345,6.41486,8000000.0
58,19851010.0,19851028.0,2.17876,7.37547,4873782.0


In [59]:
# Update the dataframe by filtering out Ghana's Longitude
gh_flood = gh_flood[gh_flood['Longitude'] < 1]
gh_flood = gh_flood[gh_flood['Longitude'] > -3]
gh_flood.head()

,StartDate,EndDate,Longitude,Latitude,Damages
973,19881001.0,19881011.0,-1.181350,9.63061,NaN
1255,19890825.0,19890829.0,-0.782037,9.49630,NaN
1809,19910714.0,19910716.0,-0.188618,6.10806,NaN
2972,19950705.0,19950707.0,-1.066620,6.43760,15000000.0
3566,19970612.0,19970615.0,0.117668,5.75584,NaN


In [60]:
# Check the dimensions of the new Ghana flood data
gh_flood.shape

(17, 5)

In [61]:
# The years where floods started in Ghana 
gh_flood.StartDate.unique()

array([19881001., 19890825., 19910714., 19950705., 19970612., 19981019.,
       20010627., 20020106., 20020610., 20090525., 20090706., 20090917.,
       20100622., 20101105., 20110722., 20111026., 20150602.])

In [62]:
# Basic statistical analysis on clean Ghana flood data
gh_flood.describe()

,StartDate,EndDate,Longitude,Latitude,Damages
count,1.700000e+01,1.700000e+01,17.000000,17.000000,1.0
mean,2.002838e+07,2.002838e+07,-0.576482,6.949248,15000000.0
std,8.560771e+04,8.560860e+04,0.419647,1.443852,NaN
min,1.988100e+07,1.988101e+07,-1.181350,5.584660,15000000.0
25%,1.997061e+07,1.997062e+07,-0.981038,5.733170,15000000.0
50%,2.002061e+07,2.002061e+07,-0.546442,6.202620,15000000.0
75%,2.010062e+07,2.010063e+07,-0.196538,7.931620,15000000.0
max,2.015060e+07,2.015062e+07,0.117668,9.630610,15000000.0


In [63]:
gh_flood = gh_flood.drop(labels=['Damages'], axis=1)

In [64]:
startdate = []
enddate = []
for i in gh_flood['StartDate']:
    s = pd.to_datetime(i, format='%Y%m%d', errors='ignore')
    startdate.append(s)
for j in gh_flood['EndDate']:
    e = pd.to_datetime(j, format='%Y%m%d', errors='ignore')
    enddate.append(e)
gh_flood.loc[:, 'StartDate'] = startdate
gh_flood.loc[:, 'EndDate'] = enddate

gh_flood

,StartDate,EndDate,Longitude,Latitude
973,1988-10-01,1988-10-11,-1.181350,9.63061
1255,1989-08-25,1989-08-29,-0.782037,9.49630
1809,1991-07-14,1991-07-16,-0.188618,6.10806
2972,1995-07-05,1995-07-07,-1.066620,6.43760
3566,1997-06-12,1997-06-15,0.117668,5.75584
3969,1998-10-19,1998-10-24,-0.150337,5.93195
4745,2001-06-27,2001-06-30,-0.290265,5.67504
4987,2002-01-06,2002-01-08,-0.196538,5.64745
5075,2002-06-10,2002-06-12,-0.196538,5.64745
7087,2009-05-25,2009-05-26,-0.099363,5.73317


In [65]:
gh_flood['year'] = pd.DatetimeIndex(gh_flood['StartDate']).year
gh_flood['month'] = pd.DatetimeIndex(gh_flood['StartDate']).month
gh_flood

,StartDate,EndDate,Longitude,Latitude,year,month
973,1988-10-01,1988-10-11,-1.181350,9.63061,1988,10
1255,1989-08-25,1989-08-29,-0.782037,9.49630,1989,8
1809,1991-07-14,1991-07-16,-0.188618,6.10806,1991,7
2972,1995-07-05,1995-07-07,-1.066620,6.43760,1995,7
3566,1997-06-12,1997-06-15,0.117668,5.75584,1997,6
3969,1998-10-19,1998-10-24,-0.150337,5.93195,1998,10
4745,2001-06-27,2001-06-30,-0.290265,5.67504,2001,6
4987,2002-01-06,2002-01-08,-0.196538,5.64745,2002,1
5075,2002-06-10,2002-06-12,-0.196538,5.64745,2002,6
7087,2009-05-25,2009-05-26,-0.099363,5.73317,2009,5
